# Importar librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
#from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score,roc_auc_score,precision_score, recall_score, f1_score,ConfusionMatrixDisplay,classification_report

# Armar un dataframe

In [2]:
df = pd.read_csv('weatherAUS.csv')
df_subset = df.iloc[:,1:]

# Vistazo rapido del dataframe

In [ ]:
df_subset.head()



In [ ]:
df_subset.tail(5)

In [ ]:
df_subset.describe()

In [ ]:
df_subset.info()

# Ver las columnas e identificar cada una

In [3]:
df_subset.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow', 'RainfallTomorrow'],
      dtype='object')

25 Columnas

**Unnamed** Indices (no esta en df_subset)

**Date** fecha de la observacion

**Location** Nombre del lugar donde esta la estacion meteorologica

**MinTemp** Temperatura minima. En grados celsius

**MaxTemp** Temperatura maxima. En grados celsius

**Rainfall** cantidad de lluvia registrada en el dia. En mm

**Evaporation** evaporacion (mm) de 00 a 09am.

**Sunshine** Numero de horas de luz solar durante el dia.

**WindGustDir** direccion de la rafaga de viento mas fuerte en las 24 horas

**WindGustSpeed** velocidad de la rafaga de viento mas fuerte en km/h

**WindDir9am** direccion del viento a las 9 am

**WindDir3pm** direccion del viento a las 3 pm

**WindSpeed9am** velocidad del viento en km/h, a las 9 am

**WindSpeed3pm** velocidad del viento en km/h, a las 3 pm

**Humidity9am** humedad en porcentaje a las 9 am

**Humidity3pm** humedad en porcentaje a las 3 pm

**Pressure9am** presion atmosferica en (hpa) al nivel del mar a las 9 am

**Pressure9am** presion atmosferica en (hpa) al nivel del mar a las 3 pm

**Cloud9am** Fraccion del cielo oscurecida por nubes medida en fracciones de 8 (0 indica sin nubes, 8 totalmente nublado) a las 9 am

**Cloud9am** Fraccion del cielo oscurecida por nubes medida en fracciones de 8 (0 indica sin nubes, 8 totalmente nublado) a las 3pm

**Temp9am** temperatura en grados celsius a las 9 am

**Temp3pm** temperatura en grados celsius a las 3 pm

**RainToday** valor booleano si llovio o no durante el dia (1 si pasa 1 mm)

**RainTomorrow** Cantidad de lluvia al dia siguiente en mm

**RainfallTomorrow** cantidad de lluvia al dia siguiente en mm

# Que no voy a usar
Date No se va a usar
Localization sera descartada ya que se tratan solamente de ciudades de Australia
tengo 145411 filas, si una columna tiene mas de 48000 valores faltantes la elimino (48000 es un tercio de los datos)
 Cloud9am y Cloud3pm tienen casi 60000 valores faltantes, Sunshine y Evaporation superan ampliamente los 48000.
 Por lo que estas columnas seran descartadas



 # Eliminar huecos en la data
tengo 145411 filas, si una columna tiene mas de 48000 valores faltantes la elimino (48000 es un tercio de los datos)
 Cloud9am y Cloud3pm tienen casi 60000 valores faltantes, Sunshine y Evaporation superan ampliamente los 48000.
 Por lo que estas columnas seran descartadas

In [4]:
df_subset = df_subset.drop(['Date', 'Location','Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm'], axis=1)

# Que valores son numericos y cuales son categoricos

In [5]:
def num_or_cat(df):
    categorical = df.select_dtypes(include='object').columns
    numerical = df.select_dtypes(exclude='object').columns

    return(categorical, numerical)

In [6]:
categorical_cols, numerical_cols= num_or_cat(df_subset)


# Contando valores faltantes

In [7]:
df_subset.isnull().sum()

MinTemp              1484
MaxTemp              1253
Rainfall             3260
WindGustDir         10316
WindGustSpeed       10253
WindDir9am          10562
WindDir3pm           4226
WindSpeed9am         1767
WindSpeed3pm         3061
Humidity9am          2653
Humidity3pm          4505
Pressure9am         15061
Pressure3pm         15024
Temp9am              1766
Temp3pm              3607
RainToday            3260
RainTomorrow         3259
RainfallTomorrow     3259
dtype: int64

# llenar columnas numericas

In [8]:
imputer = SimpleImputer(missing_values=np.nan, strategy="median", fill_value=None)

for columna in numerical_cols:
    df_subset[columna] = imputer.fit_transform(df_subset[[columna]])

In [ ]:
#knnimputer = KNNImputer(missing_values=np.nan, n_neighbors=5, weights='uniform')
#
#for columna in numerical_cols:
#    df_subset[columna] = knnimputer.fit_transform(df_subset[[columna]])

In [9]:
df_subset.isnull().sum()

MinTemp                 0
MaxTemp                 0
Rainfall                0
WindGustDir         10316
WindGustSpeed           0
WindDir9am          10562
WindDir3pm           4226
WindSpeed9am            0
WindSpeed3pm            0
Humidity9am             0
Humidity3pm             0
Pressure9am             0
Pressure3pm             0
Temp9am                 0
Temp3pm                 0
RainToday            3260
RainTomorrow         3259
RainfallTomorrow        0
dtype: int64

Capaz saque rainfall tomorrow del loop

# Cambiar Datos categoricos

En el caso de las columnas RainToday y RainTomorrow pasar Yes a 1 y no a 0

In [10]:
categorical_cols

Index(['WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow'], dtype='object')

In [ ]:
df_subset['RainToday']=df_subset['RainToday'].fillna('No')
df_subset['RainTomorrow']=df_subset['RainTomorrow'].fillna('No')


In [ ]:
df_subset['RainToday'] = df_subset['RainToday'].map({'No':0, 'Yes':1})
df_subset['RainTomorrow'] = df_subset['RainTomorrow'].map({'No':0, 'Yes':1})

# El resto de columnas categoricas reemplazar valores faltantes por la moda

In [ ]:
df_subset['WindGustDir'] = df_subset['WindGustDir'].fillna(df_subset['WindGustDir'].mode()[0])
df_subset['WindDir9am'] = df_subset['WindDir9am'].fillna(df_subset['WindDir9am'].mode()[0])
df_subset['WindDir3pm'] = df_subset['WindDir3pm'].fillna(df_subset['WindDir3pm'].mode()[0])

# ----------------------------------------------------------------------------------------------------

In [ ]:
lista = ['Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm']
numerical_cols_copy = [elemento for elemento in numerical_cols if elemento not in lista]


# Graficar datos numericos

In [ ]:
for i in range(len(numerical_cols_copy)):
    plt.figure(figsize=(7,7))
    sns.histplot(df_subset[numerical_cols_copy[i]], color="blue")
    label=numerical_cols_copy[i]
    plt.xlabel(numerical_cols_copy[i])
    plt.ylabel("count")
    plt.title(label)



In [ ]:
df_subset.isnull().sum()

# Graficar Categoricos


In [ ]:
for i in range(len(categorical_cols)):
    plt.figure(figsize=(7,7))
    sns.histplot(df_subset[categorical_cols[i]], color="blue")
    label=categorical_cols[i]
    plt.xlabel(categorical_cols[i])
    plt.ylabel("count")
    plt.title(label)

In [ ]:
df_subset.isnull().sum()

# Check si esta balanceado

In [ ]:
plt.figure(figsize=(3,3))
sns.countplot(x=df_subset['RainTomorrow'])
plt.title("Balanceado")
plt.show()

# Preparar dataset

In [ ]:
copia_df = df_subset.copy()

# Codear los datos categoricos

In [ ]:
le = LabelEncoder()
for columna in categorical_cols:
    copia_df[columna] = le.fit_transform(copia_df[columna])

In [ ]:
copia_df.head()

# Estandarizar los numericos

In [ ]:
#sc = StandardScaler()
#copia_df[numerical_cols_copy]= sc.fit_transform(copia_df[numerical_cols_copy])

In [ ]:
copia_df.head()

# Dividir en train test

In [ ]:
copia_df.head()

In [ ]:
df_subset = df_subset.drop(['Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm'], axis=1)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=69)